## GIS 5572 Lab 3 Notebook 2
Working from temperature data from Notebook 1, this notebok will do the interpolations


In [12]:
import arcpy
import requests
import json

In [13]:
arcpy.env.workspace = r'Z:\Helpdesk\Shana\GIS 5572\5572Lab3'

## Comment on workspace
I tried to have the workspace be the sde database, but that seems to make Pro very unhappy...

In [3]:
# reference the existing database created in the first notebook
weatherdata = r'Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\35.224.112.3.sde\\labgis5572.postgres.weatherfebruary_test'

## Reproject Data

In [4]:
# This code did not work. It also did not work to run it in the Geoprocessing tool but it appears to have run in the code below? 
in_dataset = "weatherdata"
out_dataset = "weatherdata_projected"
out_coor_system = 'WGS_1984_UTM_Zone_15N'
arcpy.management.Project(in_dataset, out_dataset, out_coor_system)

ExecuteError: ERROR 000622: Failed to execute (Project). Parameters are not valid.
ERROR 000628: Cannot set input into parameter out_coor_system.


In [1]:
# Why this seems to have run, I do not understand, except it wrote it to the Project folder instead of the Postgres database. Hmmmmm. 
arcpy.management.Project("Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\35.224.112.3.sde\\labgis5572.postgres.weatherfebruary_test", "projected", '32615')

<Result 'Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\5572Lab3.gdb\\projected'>

In [2]:
# This clarifies that it isn't writing to the Postgres database. I'm going to continue with the interpolations and then see if I can write those to the postgres db later
arcpy.management.Project("Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\35.224.112.3.sde\\labgis5572.postgres.weatherfebruary_test", "Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\35.224.112.3.sde\\labgis5572.postgres.projected", '32615')

ExecuteError: ERROR 000210: Cannot create output Z:\Helpdesk\Shana\GIS 5572\5572Lab3\35.224.112.3.sde\labgis5572.postgres.projected
Failed to execute (Project).


## Interpolation

In [3]:
# Testing with IDW. 
arcpy.ga.IDW('labgis5572.postgres.weatherfebruary_test', 'max_tmpf', 'idw_stats',  )

# This seemed to work. 

<Result 'idw_stats'>

## Cross Validation on the IDW
Let's try doing the Cross Validation to test it...

In [4]:
in_geostat_layer = "idw_stats"
out_point_feature_class = 'idw_max_validation'
arcpy.ga.CrossValidation(in_geostat_layer, out_point_feature_class)

<Result 'Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\idw_max_validation.shp'>

In [5]:
# After reading this documentation, I ran with the Geospatial Wizard to see what would happen
## https://pro.arcgis.com/en/pro-app/3.1/help/analysis/geostatistical-analyst/performing-cross-validation-and-validation.htm

arcpy.ga.CrossValidation(
    in_geostat_layer="idw_stats",
    out_point_feature_class=r"Z:\Helpdesk\Shana\GIS 5572\5572Lab3\idw_max_validation1.shp"
)



<Result 'Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\idw_max_validation1.shp'>

# Empirical Bayesian Kriging

In [7]:
arcpy.ga.EmpiricalBayesianKriging('projected', 'max_tmpf', 'max_kriging')

<Result 'max_kriging'>

In [ ]:
# Then ran it in the Geoprocessing tool just to see what it added. The results seem the same.

arcpy.ga.EmpiricalBayesianKriging(
    in_features="projected",
    z_field="max_tmpf",
    out_ga_layer="kriging",
    out_raster=r"Z:\Helpdesk\Shana\GIS 5572\5572Lab3\5572Lab3.gdb\kriging_raster",
    cell_size=2219.3966124,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=207149.470159255 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)

In [9]:
in_geostat_layer = "kriging"
out_point_feature_class = 'kriging_validation'
arcpy.ga.CrossValidation(in_geostat_layer, out_point_feature_class)

<Result 'Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\kriging_validation.shp'>

## Try the Exploratory Interpolation Tool

https://pro.arcgis.com/en/pro-app/latest/tool-reference/geostatistical-analyst/exploratory-interpolation.htm

In [14]:
## Starting with the code from the page above, editing it of course to meet what I want

# Interpolate points using Simple Kriging, Universal Kriging, and EBK
# Rank results by highest prediction accuracy
# Exclude results with error reductions under 25%

inPoints = "projected"
inField = "max_tmpf"
outTable = "5572Lab3.gdb\explortable"
outGALayer = "Results"
interpMethods = ["SIMPLE_KRIGING", "UNIVERSAL_KRIGING", "EBK"]
compMethod = "SORTING"
criterion = "ACCURACY"
exclCrit = [["MIN_PERC_ERROR", 25]]
arcpy.ga.ExploratoryInterpolation(inPoints, inField, outTable, outGALayer,
         interpMethods, compMethod, criterion, None, None, exclCrit)

<Result '5572Lab3.gdb\\explortable'>

In [15]:
## Try again with the same analysis, but this time save in the .sde database

inPoints = "projected"
inField = "max_tmpf"
outTable = "35.224.112.3.sde\explortable1"
outGALayer = "35.224.112.3.sde\Results1"
interpMethods = ["SIMPLE_KRIGING", "UNIVERSAL_KRIGING", "EBK", "IDW"]
compMethod = "SORTING"
criterion = "ACCURACY"
exclCrit = [["MIN_PERC_ERROR", 25]]
arcpy.ga.ExploratoryInterpolation(inPoints, inField, outTable, outGALayer,
         interpMethods, compMethod, criterion, None, None, exclCrit)

<Result '35.224.112.3.sde\\labgis5572.postgres.explortable1'>